<a href="https://colab.research.google.com/github/sriram7737/AI-Call-Center-Prototype-Faster-Whisper-TinyLLaMA-Demo/blob/main/QVLAM_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q-VLAM: Quantum-enhanced Vision-Language AI Model
This notebook combines a vision-language model (CLIP + LLaMA) with Grover's quantum search logic for enhanced prompt optimization.

In [ ]:
!pip install transformers accelerate peft torch torchvision timm  nvidia-smi qiskit==0.43.1 qiskit-aer==0.12.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 5.7 MB/s eta 0:00:00
  Created wheel for qiskit: filename=qiskit-0.43.1-py3-none-any.whl size=8144 sha256=0bfe3b3949f3acc332214ff3523986d53fc7f7e026f39336dd85b9fe436fcdcd
  Stored in directory: /root/.cache/pip/wheels/95/97/09/9b15af04f453e18fe251b496beca9c41f77dc53a8d0e978e57
Successfully built qiskit
  Attempting uninstall: symengine
    Found existing installation: symengine 0.13.0
    Uninstalling symengine-0.13.0:
      Successfully uninstalled symengine-0.13.0
  Attempting uninstall

In [ ]:
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModelForCausalLM
import torch
from PIL import Image
from qiskit_aer import Aer
from qiskit import QuantumCircuit
from qiskit.algorithms import Grover, AmplificationProblem

ModuleNotFoundError: No module named 'qiskit.algorithms'

## Load Models

In [ ]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
llama_model = AutoModelForCausalLM.from_pretrained("sriram7737/llama-3.2-1b")
tokenizer = AutoTokenizer.from_pretrained("sriram7737/llama-3.2-1b")

## Define Vision-Language Model

In [ ]:
class SimpleVisionLanguageModel(torch.nn.Module):
    def __init__(self, vision_model, text_model, projection_dim=512):
        super().__init__()
        self.vision_model = vision_model.vision_model
        self.text_model = text_model
        self.projector = torch.nn.Linear(self.vision_model.config.hidden_size, projection_dim)
        self.gate = torch.nn.Linear(projection_dim, text_model.config.hidden_size)

    def forward(self, image, input_ids, attention_mask):
        with torch.no_grad():
            vision_outputs = self.vision_model(pixel_values=image).last_hidden_state[:, 0, :]
        vision_embeds = self.projector(vision_outputs)
        gated_embeds = self.gate(vision_embeds)
        batch_size = input_ids.shape[0]
        vision_tokens = gated_embeds.unsqueeze(1).expand(batch_size, 1, -1)
        inputs_embeds = self.text_model.model.embed_tokens(input_ids)
        combined = torch.cat([vision_tokens, inputs_embeds], dim=1)
        return self.text_model(inputs_embeds=combined, attention_mask=attention_mask)

## Caption Generation Function

In [ ]:
def generate_caption(image_path, prompt="Describe the image:"):
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt")
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    attention_mask = torch.ones_like(input_ids)

    model = SimpleVisionLanguageModel(clip_model, llama_model)
    model.eval()

    with torch.no_grad():
        outputs = model(inputs['pixel_values'], input_ids, attention_mask)
        generated_ids = torch.argmax(outputs.logits, dim=-1)
        text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return text

## Quantum Search Simulation

In [ ]:
def simulate_grover(target_index=1):
    n_qubits = 2
    oracle = QuantumCircuit(n_qubits)
    oracle.x(target_index)
    oracle.cz(0, 1)
    oracle.x(target_index)
    problem = AmplificationProblem(oracle)
    grover = Grover(oracle=problem)
    backend = Aer.get_backend('aer_simulator')
    result = grover.run(backend)
    return result

## Run Example

In [ ]:
# caption = generate_caption("sample.jpg")
# print("Generated Caption:", caption)

# grover_result = simulate_grover()
# print("Grover Result:", grover_result)